# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('cruise_lr').getOrCreate()

In [4]:
data=spark.read.csv('exc/cruise_ship_info.csv', inferSchema=True, header=True)

In [55]:
# Check data schema
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [174]:
# check for nan and null values
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---------+-----------+---+-------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+-------+----------+------+------+-----------------+----+
|        0|          0|  0|      0|         0|     0|     0|                0|   0|
+---------+-----------+---+-------+----------+------+------+-----------------+----+



In [58]:
from pyspark.ml.regression import LinearRegression

In [59]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [63]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [86]:
from pyspark.ml.feature import StringIndexer

In [136]:
# Convert "Cruise_line" string column to list og caegories
str_idx=StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_indexed",
                      handleInvalid="error")

In [137]:
data_indexed=str_idx.fit(data).transform(data)

In [138]:
from pyspark.ml.feature import OneHotEncoder

In [139]:
#Assign categorical column "Cruise_line_indexed" dummy vaiables
encoder=OneHotEncoder(inputCol="Cruise_line_indexed", outputCol="Cruise_line_encoded")
data_encoded=encoder.fit(data_indexed).transform(data_indexed)
data_encoded.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_indexed: double (nullable = false)
 |-- Cruise_line_encoded: vector (nullable = true)



In [162]:
# Create feature vector for linear model (only "Cruise_line_indexed" column is used - no dummy variables)
assembler=VectorAssembler(
    inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
'Cruise_line_indexed'],
outputCol='features')

In [163]:
feature_struct=assembler.transform(data_encoded)
feature_struct.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_indexed: double (nullable = false)
 |-- Cruise_line_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [164]:
model_data=feature_struct.select('features', 'crew')
model_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [165]:
# Split data to training and test sets
train, test=model_data.randomSplit([0.85, 0.15])

In [166]:
test.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                20|
|   mean|            7.3745|
| stddev|3.3450568007135537|
|    min|               2.1|
|    max|              12.2|
+-------+------------------+



In [167]:
# Create and trian linear model
lr=LinearRegression(labelCol='crew', featuresCol='features')

In [168]:
lrModel=lr.fit(train)

In [169]:
lrModel.coefficients

DenseVector([-0.0136, 0.0031, -0.1588, 0.4431, 0.9216, 0.001, 0.0521])

In [170]:
# Evaluate model performance on test set
test_results=lrModel.evaluate(test)

In [171]:
data.select('crew').describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [172]:
test_results.meanSquaredError

0.3623663907090478

In [173]:
test_results.r2adj

0.9460253709187326